We will work on this notebook to implement the algorithm.
Please check regularly for updates.

Main tasks:
1. Write the code for the algorithm
2. Find better $\alpha$ and $\beta$ values for the model.
3. Find stopping criteria for the model.

In [15]:
# Importing the libraries:
import numpy as np
import gensim.downloader as api


In [10]:
# Toy corpus of 10 words:
toy_corpus = ['frog', 'toad', 'extrinsic', 'cat', 'intrinsic', 'dog', 'mission', 'true', 'false', 'incorrect']

In [6]:
# Load the Google News word2vec model:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [16]:
# Vectorize the toy corpus:
vec_toy_corpus = [wv[word] for word in toy_corpus]

In [30]:
# Compare vectors using cosine similarity:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Compute the cosine similarity matrix:
similarity_matrix = np.zeros((len(vec_toy_corpus), len(vec_toy_corpus)))
for i in range(len(vec_toy_corpus)):
    for j in range(len(vec_toy_corpus)):
        similarity_matrix[i][j] = cosine_similarity(vec_toy_corpus[i], vec_toy_corpus[j])

# Print the cosine similarity matrix:
for i in range(len(vec_toy_corpus)):
    for j in range(len(vec_toy_corpus)):
        print(f'Similarity between {toy_corpus[i]} and {toy_corpus[j]} is {similarity_matrix[i][j]:.4f}')

Similarity between frog and frog is 1.0000
Similarity between frog and toad is 0.7050
Similarity between frog and extrinsic is 0.0780
Similarity between frog and cat is 0.4789
Similarity between frog and intrinsic is 0.0972
Similarity between frog and dog is 0.3550
Similarity between frog and mission is 0.0185
Similarity between frog and true is 0.1128
Similarity between frog and false is -0.0008
Similarity between frog and incorrect is 0.0589
Similarity between toad and frog is 0.7050
Similarity between toad and toad is 1.0000
Similarity between toad and extrinsic is 0.0638
Similarity between toad and cat is 0.4725
Similarity between toad and intrinsic is 0.0609
Similarity between toad and dog is 0.3837
Similarity between toad and mission is 0.0607
Similarity between toad and true is 0.0903
Similarity between toad and false is 0.0358
Similarity between toad and incorrect is 0.0119
Similarity between extrinsic and frog is 0.0780
Similarity between extrinsic and toad is 0.0638
Similarit

In [ ]:
# We implement the retrofitting algorithm as a function:
